<a href="https://colab.research.google.com/github/ShikharV010/gist_daily_runs/blob/main/SalesDialierAPICall_tomorrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqlalchemy psycopg2-binary

In [ ]:
"""
JustCall Sales-Dialer ingestion
• Robust 429 handling (epoch vs. delta headers)
• Precise per-minute pacing (you choose the burst size)
• Continuous progress logging
"""

import requests, json, time, pandas as pd
from datetime import datetime, timedelta

# ────────────────────────────────────────────────────────────
# 1) CONFIG – adjust to your account
# ────────────────────────────────────────────────────────────
API_KEY    = "cc7718b616f3be5e663be9f132548cbf083fc5e9"
API_SECRET = "1f26c3c1e9bbf56324f5f9ddb70bab81b42cff38"

MAX_CALLS_PER_MIN = 28   # leave head-room under plan burst (30/60/90)
MAX_RETRIES       = 8
BACKOFF_FACTOR    = 2    # 1,2,4,8…

# Here set your target date range (e.g. tomorrow)
DATE_FROM = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
DATE_TO   = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

# ────────────────────────────────────────────────────────────
# 2) SESSION
# ────────────────────────────────────────────────────────────
session = requests.Session()
session.auth = (API_KEY, API_SECRET)             # Basic Auth
REQUEST_TIMEOUT = 15  # seconds

# ────────────────────────────────────────────────────────────
# 3) Rate-limit helpers
# ────────────────────────────────────────────────────────────
def secs_from_header(raw: str | None) -> int | None:
    if not (raw and raw.isdigit()):
        return None
    val = int(raw)
    return val - int(time.time()) if val > 86_400 else val

def respect_burst(window_start, calls_made):
    if calls_made and calls_made % MAX_CALLS_PER_MIN == 0:
        elapsed = time.time() - window_start
        wait = max(0, 60 - elapsed)
        if wait:
            print(f"↪︎ pacing – sleeping {wait:.1f}s to stay under burst limit")
            time.sleep(wait)
        return time.time()
    return window_start

# ────────────────────────────────────────────────────────────
# 4) Smart 429-handling GET & POST
# ────────────────────────────────────────────────────────────
def safe_get(url):
    for attempt in range(MAX_RETRIES):
        r = session.get(url, timeout=REQUEST_TIMEOUT)
        if r.status_code != 429:
            r.raise_for_status()
            return r.json()
        wait = secs_from_header(r.headers.get("X-Rate-Limit-Burst-Reset") or
                                r.headers.get("Retry-After"))
        wait = wait or BACKOFF_FACTOR ** attempt
        time.sleep(max(wait, 1))
    raise RuntimeError(f"gave up after {MAX_RETRIES} retries → {url}")

def safe_post(url, json_payload):
    for attempt in range(MAX_RETRIES):
        r = session.post(url, json=json_payload, timeout=REQUEST_TIMEOUT)
        if r.status_code != 429:
            r.raise_for_status()
            return r.json()
        wait = secs_from_header(r.headers.get("X-Rate-Limit-Burst-Reset") or
                                r.headers.get("Retry-After"))
        wait = wait or BACKOFF_FACTOR ** attempt
        time.sleep(max(wait, 1))
    raise RuntimeError(f"gave up after {MAX_RETRIES} retries → {url}")

# ────────────────────────────────────────────────────────────
# 5) API wrappers
# ────────────────────────────────────────────────────────────
def list_calls(date_from=DATE_FROM, date_to=DATE_TO):
    url = "https://api.justcall.io/v1/autodialer/calls/list"
    all_calls, page = [], 1
    while True:
        payload = {"start_date": date_from, "end_date": date_to, "page": page}
        resp = safe_post(url, payload)
        data = resp.get("data", [])
        if not data:
            break
        all_calls.extend(data)
        print(f"• got page {page} ({len(data)} calls)")
        page += 1
    return all_calls

def call_detail(call_id):
    url = f"https://api.justcall.io/v2.1/sales_dialer/calls/{call_id}"
    return safe_get(url).get("data", {})

# ────────────────────────────────────────────────────────────
# 6) Flatten to DataFrame
# ────────────────────────────────────────────────────────────
def flatten(detail_list):
    stamp = datetime.now().strftime("%Y-%m-%d")
    rows = []
    for d in detail_list:
        rows.append({
            "call_id":        d.get("call_id"),
            "campaign":       json.dumps(d.get("campaign", {})),
            "contact_id":     d.get("contact_id"),
            "contact_number": d.get("contact_number"),
            "contact_name":   d.get("contact_name", ""),
            "contact_email":  d.get("contact_email", ""),
            "agent_name":     d.get("agent_name", ""),
            "agent_email":    d.get("agent_email", ""),
            "call_date":      d.get("call_date"),
            "call_time":      d.get("call_time"),
            "call_info":      json.dumps(d.get("call_info", {})),
            "date_ingested":  stamp
        })
    return pd.DataFrame(rows)

# ────────────────────────────────────────────────────────────
# 7) Main workflow
# ────────────────────────────────────────────────────────────
def run_ingestion():
    print(f"\n⏳ Fetching IDs {DATE_FROM} → {DATE_TO} …")
    ids = list_calls()
    print(f"✓ {len(ids)} IDs found\n")
    details, missing = [], []
    window_start = time.time()

    for idx, c in enumerate(ids, 1):
        window_start = respect_burst(window_start, idx)
        cid = c["call_id"]
        try:
            d = call_detail(cid)
            if d:
                details.append(d)
            else:
                missing.append(cid)
            if idx % 25 == 0 or idx == len(ids):
                print(f"  progress {idx}/{len(ids)}")
        except Exception as e:
            print(f"⚠️  {cid} skipped → {e}")
            missing.append(cid)

    df = flatten(details)
    print(f"\n🏁 finished – {len(df)}/{len(ids)} rows")
    if missing:
        print(f"  still missing {len(missing)} IDs → {missing[:10]} …")
    return df

if __name__ == "__main__":
    try:
        df_calls = run_ingestion()
        # df_calls.to_csv("justcall_calls.csv", index=False)
    except Exception as err:
        print("🚨 Ingestion failed:", err)
        # optionally sys.exit(0) if you want the notebook to exit cleanly

In [ ]:
# ────────────────────────────────────────────────────────────
# 📒  Cell 2 – Enrich with city / state / time-zone (keeps the
#     original df_salesdialer unchanged; creates df_salesdialer_geo)
# ────────────────────────────────────────────────────────────

# (One-off) install if you don’t already have it in your env
# !pip install --quiet --upgrade phonenumbers

import phonenumbers
from phonenumbers import geocoder, timezone
from datetime import datetime
from zoneinfo import ZoneInfo      # Python ≥3.9
import pandas as pd                # already imported in Cell 1

# ── Helper: city / state / IANA zone from a phone number ───────────
def enrich_phone(num_str: str, default_region: str = "US") -> pd.Series:
    """
    Returns Series(city, state, iana_zone) or (None, None, None)
    """
    try:
        num_str = num_str.strip()
        # Add +1 to bare 10-digit NANP numbers so libphonenumber can parse them
        if num_str.isdigit() and len(num_str) == 10:
            num_str = "+1" + num_str

        pn = phonenumbers.parse(num_str, default_region)
        if not phonenumbers.is_possible_number(pn):
            raise ValueError("Impossible number")

        loc = geocoder.description_for_number(pn, "en")  # e.g. "Houston, TX"
        city, state = (None, None)
        if "," in loc:
            city, state = [p.strip() for p in loc.split(",", 1)]
        else:
            state = loc or None

        tzs = timezone.time_zones_for_number(pn)
        iana = tzs[0] if tzs else None

        return pd.Series([city, state, iana])

    except Exception:
        return pd.Series([None, None, None])

# ── Helper: IANA ➜ 3-letter standard-time abbreviation ────────────
def iana_to_std_abbr(iana_id: str | None) -> str | None:
    """
    'America/New_York' → 'EST'
    Uses 15 Jan so we’re always in *standard* time (avoids DST names).
    """
    if not iana_id:
        return None
    try:
        winter = datetime(2025, 1, 15)
        return winter.astimezone(ZoneInfo(iana_id)).tzname()
    except Exception:
        return None

# ── Build the enriched dataframe (original stays intact) ───────────
df_salesdialer_geo = df_calls.copy()

# ➊ Add city, state, IANA zone
df_salesdialer_geo[["city", "state", "iana_zone"]] = (
    df_salesdialer_geo["contact_number"].astype(str).apply(enrich_phone)
)

# ➋ Map to 3-letter abbrev
df_salesdialer_geo["tz_abbrev"] = (
    df_salesdialer_geo["iana_zone"].apply(iana_to_std_abbr)
)

display(df_salesdialer_geo)


In [ ]:
import pandas as pd
import sqlalchemy                       # <- new (needed only if you add dtype=)
from sqlalchemy import create_engine, text
from datetime import datetime

# ───────────── DB config ─────────────
engine = create_engine(
    "postgresql://airbyte_user:airbyte_user_password@"
    "gw-postgres-dev.celzx4qnlkfp.us-east-1.rds.amazonaws.com:5432/gw_prod"
)
TABLE_SCHEMA = "gist"
TABLE_NAME   = "gist_salesdialercalldetails"
VIEW_NAME    = "vw_salesdialercalldetails"

# ───────────── DataFrame from ingestion ─────────────
df = df_salesdialer_geo.copy()                    # <-- the only change
if df.empty:
    print("🛑 No new data to insert."); raise SystemExit

df["date_ingested"] = datetime.utcnow().date()   # keep stamp in UTC

try:
    # 1️⃣  pull existing call_ids (small result set, OK for now)
    with engine.connect() as conn:
        existing = {row[0] for row in conn.execute(
            text(f"SELECT call_id FROM {TABLE_SCHEMA}.{TABLE_NAME}")
        )}
    print(f"📦 existing rows in DB: {len(existing)}")

    # 2️⃣  filter out duplicates
    df_new = df[~df["call_id"].isin(existing)]
    print(f"🆕 rows to insert: {len(df_new)}")

    # 3️⃣  append
    if not df_new.empty:
        df_new.to_sql(
            name=TABLE_NAME,
            con=engine,
            schema=TABLE_SCHEMA,
            if_exists="append",
            index=False,
            method="multi"
            # dtype={"campaign": sqlalchemy.dialects.postgresql.JSONB,
            #        "call_info": sqlalchemy.dialects.postgresql.JSONB}
        )
        print("✅ new rows appended.")
    else:
        print("🛑 nothing new to append.")

except Exception as e:
    # table missing → create from scratch
    print(f"📭 table absent or error querying it → creating afresh.\n{e}")
    df.to_sql(
        name=TABLE_NAME,
        con=engine,
        schema=TABLE_SCHEMA,
        if_exists="replace",
        index=False,
        method="multi"
    )
    print(f"✅ table {TABLE_SCHEMA}.{TABLE_NAME} created.")

# 4️⃣  make / refresh view
with engine.begin() as conn:
    conn.execute(text(f"""
        CREATE OR REPLACE VIEW {TABLE_SCHEMA}.{VIEW_NAME} AS
        SELECT *
        FROM   {TABLE_SCHEMA}.{TABLE_NAME};
    """))
print(f"🪟 view {TABLE_SCHEMA}.{VIEW_NAME} refreshed.")
engine.dispose()
